# Test

In [1]:
import molsysmt as msm

In [2]:
import openmm as mm
from openmm import app
from openmm import unit

In [3]:
molsys = msm.convert('4POC')

In [4]:
molsys = msm.extract(molsys, selection='molecule_type=="protein"')

In [5]:
molsys = msm.build.add_missing_hydrogens(molsys)

In [6]:
molsys = msm.build.solvate(molsys)

In [7]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [8]:
topology = msm.convert(molsys, 'openmm.Topology')

In [9]:
system = forcefield.createSystem(topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)

In [10]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)

In [11]:
simulation = app.Simulation(topology, system, integrator)

In [12]:
coordinates = msm.get(molsys, coordinates=True)
positions = msm.pyunitwizard.convert(coordinates[0], to_form='openmm.unit')

In [13]:
simulation.context.setPositions(positions)
simulation.minimizeEnergy()

In [14]:
from molsysmt.thirds.openmm.reporters import MolSysMTTrajectoryDictReporter

In [15]:
reporter = MolSysMTTrajectoryDictReporter(10)

In [16]:
simulation.reporters.append(reporter)

In [17]:
simulation.step(100)

In [18]:
traj = reporter.finalize()

In [19]:
traj

{'time': Quantity(value=array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ]), unit=picosecond),
 'coordinates': Quantity(value=array([[[-0.52761507, -1.61110878, -0.43043712],
         [-0.59581929, -1.53701353, -0.42274651],
         [-0.54802489, -1.66715348, -0.51194501],
         ...,
         [-0.75094455,  2.58844328,  1.71957123],
         [ 5.33549976,  3.03071833,  5.38550663],
         [ 0.21965198, -3.46578407,  4.61602974]],
 
        [[ 2.31212519,  6.42123636, -0.42993152],
         [ 2.24291627,  6.49451229, -0.42346865],
         [ 2.29350488,  6.36354849, -0.51071763],
         ...,
         [ 7.76926383,  2.58863664,  1.71946049],
         [ 5.3344655 ,  3.03026676,  5.38542366],
         [ 3.05988586,  4.5661467 ,  4.6149044 ]],
 
        [[ 2.31282501,  6.42183921, -0.43007195],
         [ 2.24720179,  6.49855932, -0.42713368],
         [ 2.2928754 ,  6.36384901, -0.51032257],
         ...,
         [ 7.76894959,  2.58924079,  1.72070861],
     